## Trabajador experto en conocimiento

### Un agente que responde preguntas y es un trabajador experto en conocimiento
### Es un asistente que ayuda a una empresa de taxis a entender las codiciones contratuales de una póliza de seguro de vehiculos: garantías , cláusulas, primas, etc.
### El agente debe ser preciso y la solución debe ser de bajo costo.

Este proyecto utilizará RAG (Retrieval Augmented Generation, generación aumentada de recuperación) para garantizar que nuestro asistente de preguntas y respuestas tenga una alta precisión.

Esta primera implementación utilizará un tipo de RAG simple, de fuerza bruta.

In [1]:
# imports

import os
import glob
from dotenv import load_dotenv
import gradio as gr

In [2]:
# imports de langchain, plotly y Chroma

from langchain.document_loaders import DirectoryLoader, TextLoader, PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import numpy as np
import plotly.graph_objects as go
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

In [3]:
# El precio es un factor para nuestra empresa, por eso vamos a utilizar un modelo de bajo costo.

MODEL = "gpt-4o-mini"
db_name = "vector_db"

In [4]:
# Cargar variables de entorno en un archivo llamado .env

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')

In [5]:
# Obtener todas las carpetas dentro de "seguros"
folders = glob.glob("seguros/*")

def add_metadata(doc, doc_type):
    """Añade metadatos personalizados a cada documento"""
    doc.metadata["doc_type"] = doc_type
    return doc

# Cargar documentos PDF
documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    
    # Usar PyPDFLoader para PDFs - mucho mejor que TextLoader
    loader = DirectoryLoader(
        folder, 
        glob="**/*.pdf", 
        loader_cls=PyPDFLoader,
        show_progress=True  # Muestra progreso de carga
    )
    
    try:
        folder_docs = loader.load()
        documents.extend([add_metadata(doc, doc_type) for doc in folder_docs])
        print(f"✓ Cargados {len(folder_docs)} documentos de {doc_type}")
    except Exception as e:
        print(f"✗ Error cargando documentos de {folder}: {e}")

print(f"\nTotal de documentos cargados: {len(documents)}")

100%|██████████| 1/1 [00:01<00:00,  1.31s/it]

✓ Cargados 26 documentos de motor

Total de documentos cargados: 26


In [7]:
# Es más inteligente que CharacterTextSplitter para mantener contexto
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    separators=["\n\n", "\n", " ", ""]  # Intenta dividir por párrafos primero
)

chunks = text_splitter.split_documents(documents)

print(f"\nTotal de chunks generados: {len(chunks)}")
print(f"Tipos de documentos encontrados: {set(doc.metadata['doc_type'] for doc in documents)}")


Total de chunks generados: 99
Tipos de documentos encontrados: {'motor'}


In [8]:
# Coloque los fragmentos de datos en un almacén de vectores que asocie una incrustación de vectores con cada fragmento
# Chroma es una popular base de datos de vectores de código abierto basada en SQLLite

embeddings = OpenAIEmbeddings()

# Eliminar si ya existe

if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

# Crear almacén de vectores

vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)
print(f"Vectorstore created with {vectorstore._collection.count()} documents")

Vectorstore created with 99 documents


In [9]:
# Investiguemos los vectores

collection = vectorstore._collection
count = collection.count()

sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)
print(f"Hay {count:,} vectores con {dimensions:,} dimensiones en el almacén de vectores")

Hay 99 vectores con 1,536 dimensiones en el almacén de vectores


## Es hora de usar LangChain para unirlo todo

In [12]:
# create a new Chat with OpenAI
llm = ChatOpenAI(temperature=0.7, model_name=MODEL)

# set up the conversation memory for the chat
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# the retriever is an abstraction over the VectorStore that will be used during RAG
retriever = vectorstore.as_retriever()

# juntando todo: configure la cadena de conversación con GPT 3.5 LLM, el almacén vectorial y la memoria
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

In [11]:
# Vamos a intentar una pregunta sencilla

query = "Por favor, ¿qué tipo de seguro estamos contratando?."
result = conversation_chain.invoke({"question": query})
print(result["answer"])

Estamos contratando un seguro de vehículos.


In [13]:
# set up a new conversation memory for the chat
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# putting it together: set up the conversation chain with the GPT 4o-mini LLM, the vector store and memory
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

## Ahora, lo mostraremos en Gradio usando la interfaz de Chat:

Una forma rápida y sencilla de crear un prototipo de chat con un LLM

In [14]:
# Wrapping that in a function

def chat(question, history):
    result = conversation_chain.invoke({"question": question})
    return result["answer"]

In [15]:
# And in Gradio:

view = gr.ChatInterface(chat).launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
